In [2]:
!pip -q install langchain openai tiktoken chromadb

In [3]:
!pip show langchain

Name: langchain
Version: 0.1.16
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: C:\Users\satis\anaconda3\envs\OAIENV\Lib\site-packages
Requires: aiohttp, dataclasses-json, jsonpatch, langchain-community, langchain-core, langchain-text-splitters, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


In [4]:
import os
os.environ["OPENAI_API_KEY"] = ""

In [5]:
#from pydantic_settings import BaseSettings
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader

In [6]:
# Load and process the text file
# loader = TextLoader('single_file.txt')

# Loading multiple documents
loader = DirectoryLoader('./text_files',glob="./*.txt",loader_cls=TextLoader)

documents = loader.load() 

In [7]:
# splitting the text
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
texts = text_splitter.split_documents(documents)

In [8]:
len(texts)

26

In [9]:
#print the chunk of the data document
texts[4]

Document(page_content='Photography is the art, application, and practice of creating images by recording light, either electronically by means of an image sensor, or chemically by means of a light-sensitive material such as photographic film. It is employed in many fields of science, manufacturing (e.g., photolithography), and business, as well as its more direct uses for art, film and video production, recreational purposes, hobby, and mass communication.', metadata={'source': 'text_files\\Photography.txt'})

In [10]:
# Create the vector database
# Embed and store the texts


persist_directory = "database"

# using OpenAI embeddings
embedding = OpenAIEmbeddings()
vectordb = Chroma.from_documents(documents=texts,
                                 embedding=embedding,
                                 persist_directory=persist_directory)

c:\Users\satis\anaconda3\envs\oaienv\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [11]:
# persiste the data base to disk
vectordb.persist()
vectordb = None

In [12]:
# Now we can load the persisted database from the desk, and use it as normal

vectordb = Chroma(persist_directory=persist_directory,
                  embedding_function=embedding) 

In [13]:
# Make a retrivel 
retriever = vectordb.as_retriever()

In [14]:
docs = retriever.get_relevant_documents('Who is Krishna manohar IPS ?')

c:\Users\satis\anaconda3\envs\oaienv\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [15]:
len(docs)

4

In [16]:
retriever = vectordb.as_retriever(search_kwargs={"k":2})

In [17]:
retriever.search_type

'similarity'

In [18]:
retriever.search_kwargs

{'k': 2}

In [22]:
#Make a language model chain path
#Create the chain to answer questions
# https://python.langchain.com/docs/modules/data_connection/

qa_chain =RetrievalQA.from_chain_type(llm=OpenAI(),
                                      chain_type="stuff",
                                      retriever=retriever,
                                      return_source_documents=True)

In [27]:
def process_llm_response(llm_response):
    print(llm_response['result'])
    print('\n\nsources:')
    for source in llm_response['source_documents']:
        print(source.metadata['source'])

In [28]:
query = "Who is Krishna manohar IPS ?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 Krishna Manohar is an IPS officer who has gone undercover to finish off the syndicate gangs and is now a part of Ali Bhai's gang. He is the son of a retired Inspector Suryanarayana.


sources:
text_files\pokiri.txt
text_files\pokiri.txt


In [29]:
query = "How many states in USA ?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 50 states.


sources:
text_files\TheUnitedStatesofAmerica.txt
text_files\TheUnitedStatesofAmerica.txt


In [31]:
query = "Who is main characters pokiri and titanic ?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 There is no mention of characters named Pokiri or Titanic in the given context. Therefore, I am unable to provide an answer to this question.


sources:
text_files\pokiri.txt
text_files\pokiri.txt


In [32]:
query = "dance types in india ?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 There are many regional varieties of Indian classical dance, such as Odissi, Bharatanatyam, Kathak, Kuchipudi, Manipuri, and Kathakali.


sources:
text_files\Dance.txt
text_files\Dance.txt
